In [ ]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

In [ ]:
import pandas as pd
import openai
import time
from fpdf import FPDF

from google.colab import userdata

openai.api_key = userdata.get('OPENAI_KEY')

## Load the dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/noelabu/AIRepublic_Day4/refs/heads/main/dataset/manila-logistics.csv', nrows=20)

In [ ]:
df.head(5)

,delivery_id,area,neighborhood,nearest_landmark,latitude,longitude,delivery_time,traffic_condition,traffic_multiplier,parcel_type,priority,estimated_delivery_minutes,special_instructions
0,DEL000001,Makati,Poblacion,Greenbelt,14.5597,121.0284,2024-11-12 08:30:00,morning_rush,1.8,Medium Box,Express,54.0,Call upon arrival
1,DEL000002,BGC,High Street,SM Aura,14.5488,121.0519,2024-11-12 14:15:00,midday,1.2,Small Package,Standard,24.0,Leave at guard house
2,DEL000003,Quezon City,Cubao,Gateway Mall,14.6750,121.0427,2024-11-12 18:45:00,evening_rush,2.1,Large Box,Same Day,84.0,Signature required
3,DEL000004,Manila,Binondo,Lucky Chinatown,14.6005,120.9832,2024-11-12 22:30:00,night,0.9,Document Envelope,Next Day,18.0,No specific instructions
4,DEL000005,Pasig,Ortigas Center,SM Megamall,14.5774,121.0861,2024-11-13 07:45:00,morning_rush,1.9,Small Package,Express,47.5,ID required


## Understanding the Dataset

In [ ]:
# Get Column Names and Data Types
col_info = df.dtypes
print(col_info)



delivery_id                    object
area                           object
neighborhood                   object
nearest_landmark               object
latitude                      float64
longitude                     float64
delivery_time                  object
traffic_condition              object
traffic_multiplier            float64
parcel_type                    object
priority                       object
estimated_delivery_minutes    float64
special_instructions           object
dtype: object


In [ ]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)



Categorical Features: Index(['delivery_id', 'area', 'neighborhood', 'nearest_landmark',
       'delivery_time', 'traffic_condition', 'parcel_type', 'priority',
       'special_instructions'],
      dtype='object')
Numerical Features: Index(['latitude', 'longitude', 'traffic_multiplier',
       'estimated_delivery_minutes'],
      dtype='object')


In [ ]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    unique_values = df[col].unique().tolist()
    if len(unique_values) > max_display:
        unique_values = unique_values[:max_display] + ['...']
    return unique_values

unique_values_dict = {col: get_unique_values(col) for col in categorical_features}
#print(unique_values_dict)
for col, values in unique_values_dict.items():
    print(f"{col}: {', '.join(values)}")

delivery_id: DEL000001, DEL000002, DEL000003, DEL000004, DEL000005, DEL000006, DEL000007, DEL000008, DEL000009, DEL000010, ...
area: Makati, BGC, Quezon City, Manila, Pasig
neighborhood: Poblacion, High Street, Cubao, Binondo, Ortigas Center, Bel-Air, McKinley Hill, Katipunan, Malate, Kapitolyo, ...
nearest_landmark: Greenbelt, SM Aura, Gateway Mall, Lucky Chinatown, SM Megamall, Ayala Avenue, Venice Grand Canal, Trinoma, Robinsons Place, The Medical City, ...
delivery_time: 2024-11-12 08:30:00, 2024-11-12 14:15:00, 2024-11-12 18:45:00, 2024-11-12 22:30:00, 2024-11-13 07:45:00, 2024-11-13 11:20:00, 2024-11-13 17:30:00, 2024-11-13 21:15:00, 2024-11-14 09:00:00, 2024-11-14 13:45:00, ...
traffic_condition: morning_rush, midday, evening_rush, night
parcel_type: Medium Box, Small Package, Large Box, Document Envelope
priority: Express, Standard, Same Day, Next Day
special_instructions: Call upon arrival, Leave at guard house, Signature required, No specific instructions, ID required


In [ ]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print(numerical_stats)


        latitude   longitude  traffic_multiplier  estimated_delivery_minutes
count  20.000000   20.000000           20.000000                   20.000000
mean   14.591580  121.038360            1.485000                   45.500000
std     0.046847    0.034723            0.505001                   25.251368
min    14.548800  120.982200            0.800000                   16.000000
25%    14.553700  121.023150            1.050000                   22.500000
50%    14.576900  121.043700            1.500000                   43.750000
75%    14.599000  121.054150            1.925000                   60.500000
max    14.677000  121.087100            2.200000                   88.000000


## Generating the Template

In [ ]:
def generate_template(
    df,
    column_info,
    categorical_features,
    numerical_features,
    unique_values_dict,
    numerical_stats
):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in col_info.items():
        column_summary += f"{col}: {dtype}\n"


    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, values in unique_values_dict.items():
        unique_values_str += f"{col}: {', '.join(values)}\n"


    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f"- {col}\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"  {stat_name}: {value}\n"


    # Define the system prompt
    system_prompt ="""You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""


    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly.
    """

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.5,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None


In [ ]:
# Generate the template
template = generate_template(
    df,
    col_info,
    categorical_features,
    numerical_features,
    unique_values_dict,
    numerical_stats
)
print(template)


"Delivery ID {delivery_id} is scheduled for delivery to the area of {area}, specifically in the neighborhood of {neighborhood}, near the landmark {nearest_landmark}. The delivery will take place on {delivery_time}, and is expected to encounter {traffic_condition} conditions, which will apply a traffic multiplier of {traffic_multiplier}. The parcel type is classified as a {parcel_type} with a priority level of {priority}, and it is estimated to take approximately {estimated_delivery_minutes} minutes to complete the delivery. Special instructions include: {special_instructions}."


## Populating the Template with Actual Values

In [ ]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict =  row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)

    return paragraph


In [ ]:
# Apply the populate_template function to each row in the dataframe

df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

In [ ]:
df['paragraph'][0]

'"Delivery ID DEL000001 is scheduled for delivery to the Makati area, specifically in the Poblacion neighborhood, near the Greenbelt. The delivery is set for 2024-11-12 08:30:00 and will be affected by morning_rush traffic, resulting in a traffic multiplier of 1.8. The parcel type is a Medium Box with a priority level of Express. The estimated delivery time is approximately 54.0 minutes, and the special instructions include: Call upon arrival."'

## Saving the Populated Templates (Natural Language Texts)

In [ ]:
def save_content_to_txt(paragraphs ,txt_filename):
  try:
    with open(txt_filename, 'w') as file:
      for paragraph in df['paragraph']:
        file.write(paragraph + '\n')
    print(f"Content saved to {txt_filename}")
  except Exception as e:
    print(f"Error saving content to {txt_filename}: {e}")


In [ ]:
def save_content_to_pdf(paragraphs, pdf_filename):
  try:
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    for para in paragraphs:
      pdf.multi_cell(0, 10, para)
      pdf.ln()

    pdf.output(pdf_filename)
    print(f"Content saved to {pdf_filename}")
  except Exception as e:
    print(f"Error saving content to {pdf_filename}: {e}")

In [ ]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(), 'paragraphs.txt')
save_content_to_pdf(df['paragraph'].to_list(), 'paragraphs.pdf')

Content saved to paragraphs.txt
Content saved to paragraphs.pdf
